In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'pxx8jg60'

In [3]:
wandb.init(id='19fc9tj5', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

14037.137

In [5]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [7]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

CPU times: user 26.8 s, sys: 1.04 s, total: 27.8 s
Wall time: 27.8 s


In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


In [9]:
set_lr(3e-4)

In [10]:
model_stem = "11.29"
START_E = 15

In [ ]:
n_epochs = 10000
log_wandb = True

for e in range(START_E, n_epochs):
    
    lr = get_lr()
    opt.param_groups[0]['lr'] = lr if lr!=42 else (6e-4 if e < 138 else 3e-4)

    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=2560)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 15

{'lat_losses/trn_control_loss': 0.22413337, 'lat_losses/trn_headings_loss': 0.01539164, 'lat_losses/trn_curvatures_loss': 0.08042005, 'lon_losses/trn_approaching_stop_loss': 0.04404586, 'lon_losses/trn_stop_dist_loss': 0.06113369, 'lon_losses/trn_stopped_loss': 0.00016274, 'lon_losses/trn_has_lead_loss': 0.06874239, 'lon_losses/trn_lead_dist_loss': 0.02349659, 'consistency losses/trn_steer_cost': 0.04793, 'aux losses/trn_pitch_loss': 1.19412994, 'aux losses/trn_yaw_loss': 0.64394569, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.96875, 'consistency losses/trn_torque_loss': 2424.65249681, 'consistency losses/trn_torque_delta_loss': 28.13600832, 'logistical/obs_consumed_per_second': 170.88976378, 'logistical/obs_generated_per_second': 167.88188976, 'logistical/slowest_runner_obs_per_sec': 10.50393701, 'logistical/data_consumption_ratio': 1.01864885, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.21301079, 'logistical/lr': 0.0003}
{'logistical/obs_c

In [12]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m.torch")

In [13]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
